<a href="https://colab.research.google.com/github/thanhnhan311201/CS114.L11.KHCL/blob/master/SarcasmV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code : Lê Quang Huy
# Nội dung : 

- Nhận dạng văn bản trào phúng dựa vào tiêu đề.

# Ý tưởng : 
- Dùng Model Naive Bayes để  nhận dạng.

# Các bước tiến hành:
- Lấy data từ https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection.
- Đọc dữ liệu từ file json thứ nhất, chia làm 2 bộ ngẫu nhiên để train và test (Đảm bảo Dữ liệu train > Dữ liệu test)
- Lọc bỏ StopWord và các dấu chấm câu.
- Thực hiện train tương tự nội dung học trong Lớp toán cho KHMT.
- Đọc file dữ liệu thứ 2 kiểm tra accurracy.

# Kết quả thực hiện :
- Train được model .
- Accurracy 85%
- Chưa làm được Confussed Matrix 
- Chưa tính được recall và precision ...

In [ ]:
!git clone https://github.com/thanhnhan311201/CS114.L11.KHCL.git
!pip install stop-words

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.corpus import stopwords
import nltk
import json
import string

In [8]:
file_ = pd.read_json("/content/CS114.L11.KHCL/LogisticsRegression/ThucHanh/Sarcasm_Project/Sarcasm_Headlines_Dataset.json",lines=True)
del file_['article_link'] # xóa cột article_link.

# chia data thành 2 phần random để  train và test; X_test, Y_test chiếm 1/3 dữ liệu.
X_train, X_test, Y_train, Y_test = train_test_split(file_['headline'], file_['is_sarcastic'], test_size = 0.33, random_state = 50)
file_.head() # in file_

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [10]:
nltk.download('stopwords')
List_StopWord = set(stopwords.words('english')) # Danh Sach những từ sẽ xuất hiện nhiều trong câu nhưng không đóng góp quá nhiều ý nghĩa.
List_Punctuation = list(string.punctuation) # Danh sách dấu chấm câu

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def Normalize(text): # Xóa hết dấu câu cùng với stopword ra khỏi các tiêu đề.
  for x in List_StopWord:
    text.replace(x, '')
  for x in List_Punctuation:
    text.replace(x, '')
      
Normalize(X_train)

In [65]:
#Tạo Model Naive Bayes 
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter

class NaiveBayes(BaseEstimator, ClassifierMixin):
  def __init__(self):
      pass
  
  def CreateListWord(self, ListHeadLine):
    all_words = [] 
    for headline in ListHeadLine : # đọc tất cả các từ trong ListHeadLine và đưa vào all_words
      all_words.extend(headline.split()) 
    return Counter(all_words) # Trả về danh sách 1 từ đươc cuất hiện bao nhiêu lần.


  def fit(self, X, y):
    total_count = X.count() # đếm xem X có bao nhiêu dòng header -> Kích thước bộ test.
    class_counts = y.value_counts() # đếm xem có bao nhiêu kết quả cần phân loại, ở đây là 2 loại : Sarcasm :1 và Notsarcasm :0
    self.class_probs = class_counts/total_count 
    #class_probs[0] = P(NotSarcasm) 
    #class_probs[1] = P(Sarcasm)

    #chia X ra làm 2 phần 
    X_Sarcasm = X[y==1] #những headline được đánh nhãn Sarcasm 
    X_NotSarc = X[y==0] #những headline không được đánh nhãn Sarcasm 

    """Tạo 2 danh sách CountWordSarcasm và CountWordNotSar cho ta biết số lần xuất hiện của 1 từ trong X ở mỗi nhãn Sarcasm và NotSarcasm"""
    CountWordSarcasm = dict(self.CreateListWord(X_Sarcasm)) #Danh sách lưu số lần xuất hiện của 1 từ trong phần headline được đánh nhãn Sarcasm.
    CountWordNotSar  = dict(self.CreateListWord(X_NotSarc))

    CountWordSarcasm.update({k:0 for k in CountWordNotSar if k not in CountWordSarcasm})
    CountWordNotSar.update({k:0 for k in CountWordSarcasm if k not in CountWordNotSar})

    """Tính P(Xi|S) và P(Xi|NS) sau khi smoothing các giá trị"""
    CountWordSarcasm  = pd.Series(CountWordSarcasm)
    CountWordNotSar = pd.Series(CountWordNotSar)

    self.P_Xi_S = (CountWordSarcasm+1)/(CountWordSarcasm.sum() + 1)
    self.P_Xi_NS = (CountWordNotSar+1)/(CountWordNotSar.sum() + 1)
    return self
  
  def predict(self, X):
    y_pred = []
    for x in X:
      #Tính P(x|S)
      x_counts = pd.Series(self.CreateListWord([x])) #đếm những từ có trong X
      P_xi_S = self.P_Xi_S.reindex(x_counts.index) ** x_counts #Tính P(xi|S) với xi là từ xuất hiện trong X
      P_xi_S.dropna(inplace=True) #Những từ nào không xuất hiện thì bỏ qua những từ đó
      P_x_S = self.class_probs[1]*P_xi_S.product() #P(X|S) ~ Tích P(xi|S)* P(S)

      #Tính P(x|NS)
      P_xi_NS = self.P_Xi_NS.reindex(x_counts.index) ** x_counts
      P_xi_NS.dropna(inplace=True)
      P_x_NS = self.class_probs[0]*P_xi_NS.product()

      #So sánh 2 tỉ lệ P(X|S) và P(X|NS), P(X|S) lớn hơn thì dự đoán rằng đó là Sarcasm. 
      y = P_x_S >= P_x_NS 
      y_pred.append(y)
    return pd.Series(y_pred, index=X.index)


In [67]:
nb_model = NaiveBayes()
nb_model.fit(X_train, Y_train)

Normalize(X_test)
nb_model.score(X_test, Y_test)

0.8593147265713638

In [69]:
file1 = pd.read_json("/content/CS114.L11.KHCL/LogisticsRegression/ThucHanh/Sarcasm_Project/Sarcasm_Headlines_Dataset_v2.json", lines = True)
del file1['article_link']
X_test = file1['headline']
Y_test = file1['is_sarcastic']
nb_model.score(X_test, Y_test)

0.9161046856983123